In [39]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import shutil
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [40]:
# ==================== CONFIGURATION ====================
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 50
INITIAL_LR = 1e-4

In [41]:
# Paths
train_dir = '/mnt/k/ml/clg_ml/domain_classification/train'
val_dir = '/mnt/k/ml/clg_ml/domain_classification/val'
test_dir = '/mnt/k/ml/clg_ml/domain_classification/test'
MODEL_PATH = 'domain_classifier_best.h5'

In [42]:
# ==================== DATA PREPARATION ====================
def create_data_generators():
    """Create data generators with augmentation for training"""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    
    # Validation and test data generator (only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator, test_generator


In [43]:
# ==================== MODEL ARCHITECTURE ====================
def build_domain_classifier():
    """Build DenseNet121 based domain classifier"""
    
    # Load pre-trained DenseNet121
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Add custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(2, activation='softmax', name='domain_output')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model, base_model

In [44]:
# ==================== TRAINING STRATEGY ====================
def get_callbacks():
    """Define training callbacks"""
    checkpoint = ModelCheckpoint(
        MODEL_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
    
    return [checkpoint, early_stop, reduce_lr]

In [45]:
def train_domain_classifier():
    """Train the domain classifier with two-phase approach"""
    
    # Create data generators
    train_gen, val_gen, test_gen = create_data_generators()
    
    # Build model
    model, base_model = build_domain_classifier()
    
    # Phase 1: Train only the head
    print("Phase 1: Training classifier head...")
    model.compile(
        optimizer=Adam(INITIAL_LR),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history_head = model.fit(
        train_gen,
        epochs=10,
        validation_data=val_gen,
        callbacks=get_callbacks(),
        verbose=1
    )
    
    # Phase 2: Fine-tune deeper layers
    print("Phase 2: Fine-tuning deeper layers...")
    
    # Unfreeze last 50 layers of base model
    base_model.trainable = True
    for layer in base_model.layers[:-50]:
        layer.trainable = False
    
    model.compile(
        optimizer=Adam(INITIAL_LR/10),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history_full = model.fit(
        train_gen,
        epochs=EPOCHS,
        initial_epoch=history_head.epoch[-1] + 1,
        validation_data=val_gen,
        callbacks=get_callbacks(),
        verbose=1
    )
    
    return model, train_gen, val_gen, test_gen

In [46]:
# ==================== EVALUATION ====================
def evaluate_model(model, test_generator):
    """Comprehensive model evaluation"""
    
    # Get true labels and predictions
    test_generator.reset()
    y_true = test_generator.classes
    y_pred_probs = model.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    # Classification report
    class_names = list(test_generator.class_indices.keys())
    print("\n" + "="*50)
    print("CLASSIFICATION REPORT")
    print("="*50)
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("\nCONFUSION MATRIX:")
    print(cm)
    
    # Overall accuracy
    test_accuracy = np.sum(y_pred == y_true) / len(y_true)
    print(f"\nOverall Test Accuracy: {test_accuracy:.4f}")
    
    return test_accuracy, y_true, y_pred

In [47]:
# ==================== VISUALIZATION ====================
def plot_training_history(history):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [50]:
def classify_google_images(model_path, google_images_dir):
    """
    Classify images from Google images directory and show probabilities
    """
    # Load the trained model
    model = tf.keras.models.load_model(model_path)
    
    # Class names
    class_names = ['oral_disorder', 'skin_diseases']
    
    # Get all image files
    image_files = [f for f in os.listdir(google_images_dir) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp'))]
    
    print(f"Classifying {len(image_files)} images from Google directory...")
    
    for img_file in image_files:
        img_path = os.path.join(google_images_dir, img_file)
        
        # Load and preprocess image
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        
        # Make prediction
        predictions = model.predict(img_array, verbose=0)
        predicted_class = class_names[np.argmax(predictions[0])]
        confidence = np.max(predictions[0])
        
        print(f"{img_file}: Predicted → {predicted_class} ({confidence*100:.2f}%)")

In [35]:
# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("Starting Domain Classifier Training...")
    print(f"Training samples: {len(os.listdir(os.path.join(train_dir, 'skin_diseases')))} skin + {len(os.listdir(os.path.join(train_dir, 'oral_disorder')))} oral")
    print(f"Validation samples: {len(os.listdir(os.path.join(val_dir, 'skin_diseases')))} skin + {len(os.listdir(os.path.join(val_dir, 'oral_disorder')))} oral")
    
    # Train the model
    model, train_gen, val_gen, test_gen = train_domain_classifier()
    
    # Evaluate on test set
    print("\nEvaluating on test set...")
    test_accuracy, y_true, y_pred = evaluate_model(model, test_gen)
    
    # Load best model for final evaluation
    print("\nLoading best model for final evaluation...")
    best_model = tf.keras.models.load_model(MODEL_PATH)
    final_accuracy, _, _ = evaluate_model(best_model, test_gen)
    
    print(f"\n🎯 Final Domain Classifier Performance: {final_accuracy:.4f}")

Starting Domain Classifier Training...
Training samples: 4361 skin + 4789 oral
Validation samples: 531 skin + 282 oral
Found 9148 images belonging to 2 classes.
Found 813 images belonging to 2 classes.
Found 826 images belonging to 2 classes.


I0000 00:00:1759945931.275142     804 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3618 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Phase 1: Training classifier head...


/mnt/k/ml/clg_ml/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


...

286/286 ━━━━━━━━━━━━━━━━━━━━ 148s 427ms/step - accuracy: 0.9943 - loss: 0.0183 - val_accuracy: 0.9926 - val_loss: 0.0154 - learning_rate: 1.0000e-05
Epoch 12/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.9938 - loss: 0.0179
Epoch 12: val_accuracy did not improve from 0.99262
286/286 ━━━━━━━━━━━━━━━━━━━━ 93s 324ms/step - accuracy: 0.9934 - loss: 0.0195 - val_accuracy: 0.9926 - val_loss: 0.0136 - learning_rate: 1.0000e-05
Epoch 13/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.9938 - loss: 0.0171
Epoch 13: val_accuracy improved from 0.99262 to 0.99385, saving model to domain_classifier_best.h5


286/286 ━━━━━━━━━━━━━━━━━━━━ 101s 354ms/step - accuracy: 0.9938 - loss: 0.0173 - val_accuracy: 0.9938 - val_loss: 0.0103 - learning_rate: 1.0000e-05
Epoch 14/50
280/286 ━━━━━━━━━━━━━━━━━━━━ 1s 317ms/step - accuracy: 0.9908 - loss: 0.0229
Epoch 14: val_accuracy did not improve from 0.99385
286/286 ━━━━━━━━━━━━━━━━━━━━ 93s 325ms/step - accuracy: 0.9929 - loss: 0.0207 - val_accuracy: 0.9938 - val_loss: 0.0110 - learning_rate: 1.0000e-05
Epoch 15/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step - accuracy: 0.9942 - loss: 0.0144
Epoch 15: val_accuracy did not improve from 0.99385
286/286 ━━━━━━━━━━━━━━━━━━━━ 101s 354ms/step - accuracy: 0.9951 - loss: 0.0150 - val_accuracy: 0.9926 - val_loss: 0.0150 - learning_rate: 1.0000e-05
Epoch 16/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step - accuracy: 0.9974 - loss: 0.0104
Epoch 16: val_accuracy did not improve from 0.99385
286/286 ━━━━━━━━━━━━━━━━━━━━ 91s 317ms/step - accuracy: 0.9958 - loss: 0.0124 - val_accuracy: 0.9926 - val_loss: 0.0128 - learning_

26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 449ms/step

CLASSIFICATION REPORT
               precision    recall  f1-score   support

oral_disorder       0.99      1.00      0.99       282
skin_diseases       1.00      0.99      1.00       544

     accuracy                           1.00       826
    macro avg       0.99      1.00      0.99       826
 weighted avg       1.00      1.00      1.00       826


CONFUSION MATRIX:
[[282   0]
 [  4 540]]

Overall Test Accuracy: 0.9952

🎯 Final Domain Classifier Performance: 0.9952


Classifying 8 images from Google directory...


NameError: name 'image' is not defined

In [52]:
classify_google_images('domain_classifier_best.h5', '/mnt/k/ml/clg_ml/imgs_from_google/')

Classifying 12 images from Google directory...
be_ke_le.jpeg: Predicted → skin_diseases (100.00%)
be_ke_le2.jpg: Predicted → skin_diseases (100.00%)
ecz.webp: Predicted → skin_diseases (98.70%)
ecz2.jpg: Predicted → skin_diseases (99.31%)
hypo.jpeg: Predicted → oral_disorder (100.00%)
hypo2.jpg: Predicted → oral_disorder (100.00%)
hypo3.jpg: Predicted → oral_disorder (99.90%)
mo_ul.jpeg: Predicted → oral_disorder (100.00%)
mo_ul2.jpg: Predicted → oral_disorder (99.99%)
mo_ul3.jpg: Predicted → oral_disorder (99.97%)
pso.webp: Predicted → skin_diseases (99.99%)
pso2.jpg: Predicted → skin_diseases (100.00%)
